## Imports

In [79]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
%matplotlib inline

In [80]:
# define all dataframes for later

# gross profits from Box Office Movies (an IMDB subsidiary)
# gross_df = pd.read_csv('data/zipped_files/bom.movie_gross.csv.gz')

# the movie database movie database
# movies_df = pd.read_csv('data/zipped_files/tmdb.movies.csv.gz')

# movie budgets and gross profits from The Numbers
budgets_df = pd.read_csv('data/zipped_files/tn.movie_budgets.csv.gz')

### vvv probably a 99% chance these ones don't get used vvv ###

# tsv files have to be read with \t (tab spcaes) as their delimiter
# info_df = pd.read_csv('data/zipped_files/rt.movie_info.tsv.gz',delimiter='\t')
# file has an issue with encoding, read as cp850 or it throws an error
# reviews_df = pd.read_csv('data/zipped_files/rt.reviews.tsv.gz', delimiter='\t', encoding='cp850')

### IM.db database structure
![database schema flow chart](./images/db_schema.jpeg)

In [81]:
#################################
# REQUIRES UNZIPPING data/zipped_files/im.db.zip
# INTO DIRECOTRY data/unzipped AS im.db
#################################
conn = sqlite3.connect('data/unzipped/im.db')
# call this later => imdb_df = pd.read_sql('''<QUERY>''',conn)

In [82]:
### GETTING NAMES OF ALL TABLE NAMES, USED TO REFERENCE IN-LINE QUERIES
tables = pd.read_sql(
"""
SELECT * FROM sqlite_master
""", conn)

In [83]:
# create imdb_directors with director names and movie IDs
imdb_directors = pd.read_sql("""
SELECT d.movie_id, p.primary_name FROM directors as d
LEFT JOIN persons AS p
    ON d.person_id = p.person_id
GROUP BY d.movie_id
""", conn)

In [84]:
# attach each possible movie title with their respective movie id,
# lets us look up the potentially mismatched titles from budgets_df
# doesn't need to be distinguished into original_ or primary_title
# beyond this
movie_akas = pd.read_sql("""
SELECT DISTINCT movie_id, title FROM movie_akas
""",conn)

In [85]:
# list-ifys the movie titles and movie IDs 
movie_akas_list = list(movie_akas['title'])
movie_ids_list = list(movie_akas['movie_id'])

In [86]:
# convert budgets_df['release_date'] to date time object and gets the 
budgets_df['clean_release_date'] =  pd.to_datetime(budgets_df['release_date'], infer_datetime_format=True).dt.year
budgets_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,clean_release_date
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279",2009
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",2011
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",2019
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",2015
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747",2017


In [87]:
budgets_df['movie_id'] = [movie_ids_list[movie_akas_list.index( title )] if title in movie_akas_list else None for title in budgets_df['movie']]

In [88]:
# get rid of entries without a movie_id - director can't be found without this info
budgets_df.dropna(subset=['movie_id'],inplace=True)

In [89]:
# remove duplicitous entries of movie_id, 1 movie = 1 id
budgets_df.drop_duplicates(subset=['movie_id'],inplace=True)

In [90]:
# clean budgets_df to recent movies for comparisons
recent_budgets_df = budgets_df[budgets_df['clean_release_date'] >= 2013]
recent_budgets_df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,clean_release_date,movie_id
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",2019,tt6565702
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",2015,tt2395427
6,7,"Apr 27, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,134,200",2018,tt4154756
8,9,"Nov 17, 2017",Justice League,"$300,000,000","$229,024,295","$655,945,209",2017,tt0974015
9,10,"Nov 6, 2015",Spectre,"$300,000,000","$200,074,175","$879,620,923",2015,tt2379713
...,...,...,...,...,...,...,...,...
5755,56,"Jan 4, 2013",All Superheroes Must Die,"$20,000",$0,$0,2013,tt1836212
5756,57,"Apr 21, 2015",The Front Man,"$20,000",$0,$0,2015,tt2357398
5761,62,"Dec 31, 2014",Stories of Our Lives,"$15,000",$0,$0,2014,tt3973612
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0,2018,tt7837402


In [91]:
# get movies from im.db that are from 2013 and newer to base trends on recency
recent_imdb_movies = pd.read_sql("""
SELECT * FROM movie_basics
WHERE CAST(start_year AS int) BETWEEN 2013 AND 2023
""", conn)

In [92]:
recent_imdb_movies.sort_values('start_year')

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
17314,tt2971400,Loving Memory,Loving Memory,2013,NaN,Drama
17313,tt2971388,308,308,2013,121.0,"Horror,Mystery,Thriller"
17312,tt2971128,"River Deep, Mountain High: James Nesbitt in Ne...","River Deep, Mountain High: James Nesbitt in Ne...",2013,NaN,Documentary
17311,tt2970792,Wechselspiel,Wechselspiel,2013,73.0,Drama
...,...,...,...,...,...,...
5999,tt1757678,Avatar 3,Avatar 3,2023,NaN,"Action,Adventure,Drama"
74209,tt6495056,Untitled Illumination Entertainment Project,Untitled Illumination Entertainment Project,2023,NaN,None
2392,tt10298848,Untitled Disney Live-Action Project,Untitled Disney Live-Action Project,2023,NaN,None
70264,tt6258542,Wraith of the Umbra and Eidolon II,Wraith of the Umbra and Eidolon II,2023,NaN,"Adventure,Drama,Fantasy"


In [93]:
budgets_df.dropna(subset=["movie_id"],axis=0,inplace=True)

In [94]:
# convert dollar amounts (str) into int
budgets_df['domestic_gross'] = budgets_df['domestic_gross'].str.replace('$', '').str.replace(',', '').astype(int)
budgets_df['worldwide_gross'] = budgets_df['worldwide_gross'].str.replace('$', '').str.replace(',', '').astype(int)
budgets_df['production_budget'] = budgets_df['production_budget'].str.replace('$', '').str.replace(',', '').astype(int)

# calculate foreign gross based off worldwide - domestic values
budgets_df['foreign_gross'] = budgets_df['worldwide_gross'] - budgets_df['domestic_gross']

In [95]:
budgets_df.shape

(2406, 9)

In [96]:
budgets_df.merge(recent_imdb_movies, on="movie_id", how='left').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2406 entries, 0 to 2405
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  2406 non-null   int64  
 1   release_date        2406 non-null   object 
 2   movie               2406 non-null   object 
 3   production_budget   2406 non-null   int64  
 4   domestic_gross      2406 non-null   int64  
 5   worldwide_gross     2406 non-null   int64  
 6   clean_release_date  2406 non-null   int64  
 7   movie_id            2406 non-null   object 
 8   foreign_gross       2406 non-null   int64  
 9   primary_title       1465 non-null   object 
 10  original_title      1465 non-null   object 
 11  start_year          1465 non-null   float64
 12  runtime_minutes     1375 non-null   float64
 13  genres              1460 non-null   object 
dtypes: float64(2), int64(6), object(6)
memory usage: 282.0+ KB


In [97]:
# removing any entries that have a $0 gross worldwide - most likely missing data, hundreds of entries
budgets_df = budgets_df[budgets_df['worldwide_gross'] > 0]

In [76]:
recent_imdb_movies

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
107597,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
107598,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
107599,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
107600,tt9916730,6 Gunn,6 Gunn,2017,116.0,None


In [78]:
budgets_df.shape

(35, 9)

In [74]:
movies_with_gross_df = budgets_df.merge(recent_imdb_movies, on="movie_id", how='inner')
movies_with_gross_df.shape #[['movie','production_budget','domestic_gross','worldwide_gross','movie_id',]]

(31, 14)

In [27]:
# fixed_titles.dropna(subset=['movie_id']).sort_values('domestic_gross',ascending=False)

In [324]:
# movie_akas.groupby(by='movie_id').agg(lambda x: list(x))